In [1]:
from tensorflow import keras
import numpy as np
import tensorflow as tf

In [3]:
train_data = tf.keras.utils.image_dataset_from_directory(
    directory='./dataset/train',
    labels="inferred",
    image_size = (224, 224),
    label_mode='categorical',
    batch_size=16,
    shuffle=True
)
# train_data = train_data.map(lambda x, y: (tf.image.resize(x, [128, 128]), y)).shuffle(1024)

NotFoundError: Could not find directory ./dataset/train

In [16]:
import os
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img

DIRECTORY = "./dataset"
CATEGORIES = ["with_mask", "without_mask", "mask_weared_incorrect"]
######################################################################

data = []
labels = []

for category in CATEGORIES:
    path = os.path.join(DIRECTORY, category)
    for img in os.listdir(path):
    	img_path = os.path.join(path, img)
        
    	image = load_img(img_path, target_size=(224, 224))
    	image = img_to_array(image)
        data = np.reshape(image ,[224,224,3])/255.0

    	data.append(image)
    	labels.append(category)

TabError: inconsistent use of tabs and spaces in indentation (2941098475.py, line 19)

In [15]:
data = np.reshape(data ,[224,224,3])/255.0

ValueError: cannot reshape array of size 1833581568 into shape (224,224,3)

In [ ]:
# phân chia dữ liệu train:test(80:20), test:val(70:30)
from sklearn.model_selection import train_test_split

train_with_mask, test_with_mask = train_test_split(with_mask, test_size=0.20, random_state=42)
test_with_mask, val_with_mask = train_test_split(test_with_mask, test_size=0.7, random_state=42)

train_mask_weared_incorrect, test_mask_weared_incorrect = train_test_split(mask_weared_incorrect, test_size=0.20, random_state=42)
test_mask_weared_incorrect, val_mask_weared_incorrect = train_test_split(test_mask_weared_incorrect, test_size=0.7, random_state=42)

train_without_mask, test_without_mask = train_test_split(without_mask, test_size=0.20, random_state=42)
test_without_mask, val_without_mask = train_test_split(test_without_mask, test_size=0.7, random_state=42)

In [ ]:
train_data

In [ ]:
test_data = tf.keras.utils.image_dataset_from_directory(
    directory='./dataset/test',
    labels="inferred",
    image_size = (224, 224),
    label_mode='categorical',
    batch_size=16,
    shuffle=True
)
# test_data = test_data.map(lambda x, y: (tf.image.resize(x, [128, 128]), y)).shuffle(1024)

In [ ]:
val_data = tf.keras.utils.image_dataset_from_directory(
    directory='./dataset/val',
    labels="inferred",
    image_size = (224, 224),
    label_mode='categorical',
    batch_size=16,
    shuffle=True
)

In [ ]:
batch_size = 16
epochs = 100

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
val_datagen = ImageDataGenerator(
    rescale=1.0 / 255
)

datagen = ImageDataGenerator(
    rescale=1.0 / 255, #chuẩn hóa (0,1)
    horizontal_flip=True, #lật theo chiều dọc
    zoom_range=0.1, #phóng to 10%
    shear_range=0.2, # làm méo ảnh
    width_shift_range=0.1, #dịch chuyển ngang 10%
    height_shift_range=0.1, #dịch chuyển dọc 10%
    rotation_range=4, #xoay 4 độ
    vertical_flip=False 

)
 
# train data
train_generator = datagen.flow_from_directory(
    directory='./kaggle/working/train', 
    target_size = (224,224),
    class_mode="categorical",
    batch_size=batch_size,
    shuffle=True

)

# Validation data
val_generator = val_datagen.flow_from_directory(
    directory='./kaggle/working/val', 
    target_size = (224,224),
    class_mode="categorical", 
    batch_size=batch_size, 
    shuffle=True
)

# Test data
test_generator = val_datagen.flow_from_directory(
    directory='./kaggle/working/test', 
    target_size = (224,224),
    class_mode="categorical", 
    batch_size=batch_size, 
    shuffle=False
)

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model_custom = Sequential()
model_custom.add(Conv2D(filters = 16, kernel_size = 3,  padding='same', activation = 'relu', input_shape = (224,224,3)))
model_custom.add(MaxPooling2D(pool_size = 2))
model_custom.add(Conv2D(filters = 32, kernel_size = 3,  padding='same', activation = 'relu'))
model_custom.add(MaxPooling2D(pool_size = 2))
model_custom.add(Conv2D(filters = 64, kernel_size = 3,  padding='same', activation = 'relu'))
model_custom.add(MaxPooling2D(pool_size = 2))
model_custom.add(Dropout(0.3))
model_custom.add(Flatten())
model_custom.add(Dense(units = 500, activation = 'relu'))
model_custom.add(Dropout(0.3))
model_custom.add(Dense(units = 3, activation = 'softmax'))


model_custom.summary()

In [ ]:
data_size = len(train_data) 
#data_size2 = train_generator.n

#print(f"data_size: {data_size}, {data_size2}")

steps_per_epoch = int(data_size / batch_size)
print(f"steps_per_epoch: {steps_per_epoch}")

val_steps = int(len(val_data) // batch_size)
#print(f"val size: {len(val_generator)}")
print(f"val_steps: {val_steps}")

In [ ]:
# Compiling the model
import tensorflow as tf
model_custom.compile(
    optimizer="adam",
    loss= tf.keras.losses.CategoricalFocalCrossentropy(
        gamma=1.5,
        name='categorical_focal_crossentropy'
    ),
    metrics=['accuracy', 'Recall', 'Precision', 'AUC']

)

# Early Stopping
# Before training the network, we define an early stopping criterion,
#to avoid redundent epochs once the model has already converged.
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)

# Reduce Learning Rate On Plateau
# We define a `ReduceLROnPlateau` callback to reduce the learning rate when the metric
# we chose (`val_loss`) has stopped improving.
from tensorflow.keras.callbacks import ReduceLROnPlateau
lrr = ReduceLROnPlateau(monitor='val_loss',patience=8,verbose=1,factor=0.5, min_lr=0.00001)

In [ ]:
model_custom_history = model_custom.fit_generator(
    generator=train_data,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    shuffle=True,
    #verbose=2,
    validation_data=val_data,
    validation_steps=val_steps,
    callbacks=[lrr]
)

model_custom.save('model_custom3.h5')

In [ ]:
# Evaluate model performance on test data
model_custom_loss, model_custom_acc, custom_recall, custom_precision, custom_auc = model_custom.evaluate(test_generator)
print("Our Custom Model has a loss of %.2f and accuracy %.2f%%" % (model_custom_loss, model_custom_acc*100))
print("Our Custom Model has a recall of %.2f%%, precision of %.2f%% and auc of %.2f%%" % (custom_recall*100, custom_precision*100, custom_auc*100))

In [ ]:
custom_predictions = model_custom.predict(test_generator)
print("predictions shape:", custom_predictions.shape)

In [ ]:
import cv2

paths = test_generator.filenames # mảng chứa đường dẫn của tên file test
y_pred_custom = model_custom.predict(test_generator).argmax(axis=1) # mảng chứa dự đoán tất cả nhãn
classes = test_generator.class_indices  # danh sách tên nhãn

a_img_rand = np.random.randint(0,len(paths))   # chọn ngẫu nhiên thứ tự hình ảnh
img = cv2.imread(os.path.join(output_data_path,'test', paths[a_img_rand]))  
colored_img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)  # ảnh đầu ra có màu

# xuất hình ảnh được chọn ngẫu nhiên
plt.imshow(colored_img)
# nhãn đúng
true_label = paths[a_img_rand].split('/')[0]
# dự đoán nhãn
predicted_label = list(classes)[y_pred_custom[a_img_rand]]
print(f'Class Predicted: {predicted_label} , True label: {true_label} for Custom Model')

In [ ]:
y_true = test_generator.labels
y_pred_custom  = model_custom.predict(test_generator).argmax(axis=1) # Predict prob and get Class Indices

In [ ]:
import cv2
img = cv2.imread(os.path.join(input_data_path, f'{final_test_image}.png'))      
colored_img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)  # colored output image

#print(type(colored_img))
mask_label = {0:'MASK INCORRECT',1:'MASK', 2:'NO MASK'}
color_label = {0:(0,255,255),1:(0, 255,0), 2:(255,0,0)}
cropped_faces = extract_faces(f'{final_test_image}.png', df_final_test)

df_test_img = df_final_test[['xmin', 'ymin', 'xmax', 'ymax', 'label']]
#for row_num in range(len(df_test_img)):
for idx, face in enumerate(cropped_faces):    
    # print(idx)
    x_min, y_min, x_max, y_max, label = df_test_img.iloc[idx] 
    # print(x_min, y_min, x_max, y_max, label)
   
    resized_face = cv2.resize(np.array(face[0]),(224,224))
    # chuẩn hóa
    reshaped_face = np.reshape(resized_face,[1,224,224,3])/255.0
    # mảng lưu trũ tỉ lệ dự đoán của từng nhãn
    face_result = model_custom.predict(reshaped_face)

    cv2.putText(colored_img,mask_label[face_result.argmax()],(x_min, y_min-10),cv2.FONT_HERSHEY_SIMPLEX,0.5,color_label[face_result.argmax()],2)
    cv2.rectangle(colored_img,(x_min, y_min), (x_max, y_max), color_label[face_result.argmax()]) # print a blue rectangle of each person's face using the given coordinates

plt.figure(figsize=(10, 10))    
plt.imshow(colored_img)